In [3]:
import cv2
import argparse
import numpy as np
import os.path
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib import pyplot as plt
%matplotlib inline

from bokeh.plotting import figure #pip install bokeh
from bokeh.io import output_notebook, show, push_notebook


output_notebook()


def imshow(tit, image) :
    plt.title(tit)    
    if len(image.shape) == 3 :
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    else :
        plt.imshow(image, cmap="gray")
    plt.show()
    
    
def create_win(frames, scale=1.0) :    
    global myImage
    
    all = []
    for f in frames :
        if len(f.shape ) !=  3 : f = cv2.cvtColor(f, cv2.COLOR_GRAY2BGR)
        all.append(f)
    frame = np.vstack(all)
    
    fr=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
    fr=cv2.flip(fr, -1) # because Bokeh flips vertically
    width=fr.shape[1]
    height=fr.shape[0]    

    p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=int(width*scale), height=int(height*scale))    
    myImage = p.image_rgba(image=[fr], x=0, y=0, dw=width, dh=height)
    show(p, notebook_handle=True)   
    
    
def update_win(frames) :
    
    all = []
    for f in frames :
        #print(len(f.shape))
        if len(f.shape ) !=  3 : f = cv2.cvtColor(f, cv2.COLOR_GRAY2BGR)
        all.append(f)
    frame = np.vstack(all)
    
    fr=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    fr=cv2.flip(fr, 0)
    myImage.data_source.data['image']=[fr]
    push_notebook()
    # Initialize the parameters
confThreshold = 0.3  #Confidence threshold, 오브젝트일 확률
nmsThreshold = 0.4   #Non-maximum suppression threshold , 거리가 가까운애들은 하나의 그룹으로 보겟다, 주변보다 상대적으로 값이 높거나 낮거나 하는 애를 뽑겠다.

#YOLO v3는 영상을 무조건 416 사이즈로 변경
inpWidth = 416       #Width of network's input image
inpHeight = 416      #Height of network's input image

# Load names of classes
classesFile = "cfg/one.names" 
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')
print(classes)




# Get the names of the output layers
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames() #네트웍의 모든 이름을 가져오는 함수 ,총 갯수는 254개, 실제 레이어는 100 몇개,
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()] #실제 output 위치는 getUnconnectedOutLayers에서 1뺌

# Draw the predicted bounding box
#사각형으로 바운딩한 곳을 표시
def drawPred(frame, classId, conf, left, top, right, bottom): #클래스ID, 클래스에 대한 일치 확률,사각형 정보
    # Draw a bounding box.
    cv2.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
    
    label = '%.2f' % conf
        
    # Get the label for the class name and its confidence
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)

    #Display the label at the top of the bounding box
    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv2.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (255, 255, 255), cv2.FILLED)
    cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1)

# Remove the bounding boxes with low confidence using non-maxima suppression
#
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:] #확률 80개의 값을 가져옴
            classId = np.argmax(scores) #확률에서 높은거 가져옴
            confidence = scores[classId] #확률 값 가져옴
            if confidence > confThreshold: # confThreshold보다 더 높을 경우, 박스를 만듬
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
        
    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    #같은 포인트에서 다수의 바운딩 박스가 생성되어 있는걸 좀더 정확하게 판별해서 최소화 해줌
    
    for i in indices:
        i = i[0] #2차원 행렬이므로 
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        drawPred(frame, classIds[i], confidences[i], left, top, left + width, top + height)
        
# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = "cfg/one.cfg" #네트워크 구조체(CNN 필터 값들)가 포함되어있다, #컨피그 상에서 실제로 Yolo 관련 컨피그가 3개가 있다.
modelWeights = "cfg/one_class_v1_1800.weights"
        

net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
        
print("YOLO Video process")
cap = cv2.VideoCapture("video/102-5_cam02_swoon01_place02_night_spring.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
print('width :%d, height : %d' % (frame_width, frame_height))
fourcc = cv2.VideoWriter_fourcc(*'XVID') #운영체제별로 지원하는게 다름 확인필요
out = cv2.VideoWriter('video_result/102-5_cam02_swoon01_place02_night_spring_1800.avi', fourcc, 29.0, (frame_width, frame_height)) 
i = 0
while(True):
    hasFrame, frame = cap.read()    # Read 결과와 frame
    if not hasFrame:
        break
    #print(hasFrame)
    #print(frame)
    print(i)
    if(hasFrame) :
        #if i > 5000:
        print(i, ': caputer ..')
        blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)
        net.setInput(blob)
        outs = net.forward(getOutputsNames(net))
        postprocess(frame, outs)
        #cv2.imshow('frame_color', frame)    # 컬러 화면 출력
        out.write(frame)
    i += 1

cap.release()
cv2.destroyAllWindows()

Loading BokehJS ...

['fainting_people']
YOLO Video process
width :3840, height : 2160
0
0 : caputer ..
1
1 : caputer ..
2
2 : caputer ..
3
3 : caputer ..
4
4 : caputer ..
5
5 : caputer ..
6
6 : caputer ..
7
7 : caputer ..
8
8 : caputer ..
9
9 : caputer ..
10
10 : caputer ..
11
11 : caputer ..
12
12 : caputer ..
13
13 : caputer ..
14
14 : caputer ..
15
15 : caputer ..
16
16 : caputer ..
17
17 : caputer ..
18
18 : caputer ..
19
19 : caputer ..
20
20 : caputer ..
21
21 : caputer ..
22
22 : caputer ..
23
23 : caputer ..
24
24 : caputer ..
25
25 : caputer ..
26
26 : caputer ..
27
27 : caputer ..
28
28 : caputer ..
29
29 : caputer ..
30
30 : caputer ..
31
31 : caputer ..
32
32 : caputer ..
33
33 : caputer ..
34
34 : caputer ..
35
35 : caputer ..
36
36 : caputer ..
37
37 : caputer ..
38
38 : caputer ..
39
39 : caputer ..
40
40 : caputer ..
41
41 : caputer ..
42
42 : caputer ..
43
43 : caputer ..
44
44 : caputer ..
45
45 : caputer ..
46
46 : caputer ..
47
47 : caputer ..
48
48 : caputer ..
49
49 : caputer ..
50
5

398
398 : caputer ..
399
399 : caputer ..
400
400 : caputer ..
401
401 : caputer ..
402
402 : caputer ..
403
403 : caputer ..
404
404 : caputer ..
405
405 : caputer ..
406
406 : caputer ..
407
407 : caputer ..
408
408 : caputer ..
409
409 : caputer ..
410
410 : caputer ..
411
411 : caputer ..
412
412 : caputer ..
413
413 : caputer ..
414
414 : caputer ..
415
415 : caputer ..
416
416 : caputer ..
417
417 : caputer ..
418
418 : caputer ..
419
419 : caputer ..
420
420 : caputer ..
421
421 : caputer ..
422
422 : caputer ..
423
423 : caputer ..
424
424 : caputer ..
425
425 : caputer ..
426
426 : caputer ..
427
427 : caputer ..
428
428 : caputer ..
429
429 : caputer ..
430
430 : caputer ..
431
431 : caputer ..
432
432 : caputer ..
433
433 : caputer ..
434
434 : caputer ..
435
435 : caputer ..
436
436 : caputer ..
437
437 : caputer ..
438
438 : caputer ..
439
439 : caputer ..
440
440 : caputer ..
441
441 : caputer ..
442
442 : caputer ..
443
443 : caputer ..
444
444 : caputer ..
445
445 : cap

789
789 : caputer ..
790
790 : caputer ..
791
791 : caputer ..
792
792 : caputer ..
793
793 : caputer ..
794
794 : caputer ..
795
795 : caputer ..
796
796 : caputer ..
797
797 : caputer ..
798
798 : caputer ..
799
799 : caputer ..
800
800 : caputer ..
801
801 : caputer ..
802
802 : caputer ..
803
803 : caputer ..
804
804 : caputer ..
805
805 : caputer ..
806
806 : caputer ..
807
807 : caputer ..
808
808 : caputer ..
809
809 : caputer ..
810
810 : caputer ..
811
811 : caputer ..
812
812 : caputer ..
813
813 : caputer ..
814
814 : caputer ..
815
815 : caputer ..
816
816 : caputer ..
817
817 : caputer ..
818
818 : caputer ..
819
819 : caputer ..
820
820 : caputer ..
821
821 : caputer ..
822
822 : caputer ..
823
823 : caputer ..
824
824 : caputer ..
825
825 : caputer ..
826
826 : caputer ..
827
827 : caputer ..
828
828 : caputer ..
829
829 : caputer ..
830
830 : caputer ..
831
831 : caputer ..
832
832 : caputer ..
833
833 : caputer ..
834
834 : caputer ..
835
835 : caputer ..
836
836 : cap

1164
1164 : caputer ..
1165
1165 : caputer ..
1166
1166 : caputer ..
1167
1167 : caputer ..
1168
1168 : caputer ..
1169
1169 : caputer ..
1170
1170 : caputer ..
1171
1171 : caputer ..
1172
1172 : caputer ..
1173
1173 : caputer ..
1174
1174 : caputer ..
1175
1175 : caputer ..
1176
1176 : caputer ..
1177
1177 : caputer ..
1178
1178 : caputer ..
1179
1179 : caputer ..
1180
1180 : caputer ..
1181
1181 : caputer ..
1182
1182 : caputer ..
1183
1183 : caputer ..
1184
1184 : caputer ..
1185
1185 : caputer ..
1186
1186 : caputer ..
1187
1187 : caputer ..
1188
1188 : caputer ..
1189
1189 : caputer ..
1190
1190 : caputer ..
1191
1191 : caputer ..
1192
1192 : caputer ..
1193
1193 : caputer ..
1194
1194 : caputer ..
1195
1195 : caputer ..
1196
1196 : caputer ..
1197
1197 : caputer ..
1198
1198 : caputer ..
1199
1199 : caputer ..
1200
1200 : caputer ..
1201
1201 : caputer ..
1202
1202 : caputer ..
1203
1203 : caputer ..
1204
1204 : caputer ..
1205
1205 : caputer ..
1206
1206 : caputer ..
1207
1207 :

...

8661
8661 : caputer ..
8662
8662 : caputer ..
8663
8663 : caputer ..
8664
8664 : caputer ..
8665
8665 : caputer ..
8666
8666 : caputer ..
8667
8667 : caputer ..
8668
8668 : caputer ..
8669
8669 : caputer ..
8670
8670 : caputer ..
8671
8671 : caputer ..
8672
8672 : caputer ..
8673
8673 : caputer ..
8674
8674 : caputer ..
8675
8675 : caputer ..
8676
8676 : caputer ..
8677
8677 : caputer ..
8678
8678 : caputer ..
8679
8679 : caputer ..
8680
8680 : caputer ..
8681
8681 : caputer ..
8682
8682 : caputer ..
8683
8683 : caputer ..
8684
8684 : caputer ..
8685
8685 : caputer ..
8686
8686 : caputer ..
8687
8687 : caputer ..
8688
8688 : caputer ..
8689
8689 : caputer ..
8690
8690 : caputer ..
8691
8691 : caputer ..
8692
8692 : caputer ..
8693
8693 : caputer ..
8694
8694 : caputer ..
8695
8695 : caputer ..
8696
8696 : caputer ..
8697
8697 : caputer ..
8698
8698 : caputer ..
8699
8699 : caputer ..
8700
8700 : caputer ..
8701
8701 : caputer ..
8702
8702 : caputer ..
8703
8703 : caputer ..
8704
8704 :

9018
9018 : caputer ..
9019
9019 : caputer ..
9020
9020 : caputer ..
9021
9021 : caputer ..
9022
9022 : caputer ..
9023
9023 : caputer ..
9024
9024 : caputer ..
9025
9025 : caputer ..
9026
9026 : caputer ..
9027
9027 : caputer ..
9028
9028 : caputer ..
9029
9029 : caputer ..
9030
9030 : caputer ..
9031
9031 : caputer ..
9032
9032 : caputer ..
9033
9033 : caputer ..
9034
9034 : caputer ..
9035
9035 : caputer ..
9036
9036 : caputer ..
9037
9037 : caputer ..
9038
9038 : caputer ..
9039
9039 : caputer ..
9040
9040 : caputer ..
9041
9041 : caputer ..
9042
9042 : caputer ..
9043
9043 : caputer ..
9044
9044 : caputer ..
9045
9045 : caputer ..
9046
9046 : caputer ..
9047
9047 : caputer ..
9048
9048 : caputer ..
9049
9049 : caputer ..
9050
9050 : caputer ..
9051
9051 : caputer ..
9052
9052 : caputer ..
9053
9053 : caputer ..
9054
9054 : caputer ..
9055
9055 : caputer ..
9056
9056 : caputer ..
9057
9057 : caputer ..
9058
9058 : caputer ..
9059
9059 : caputer ..
9060
9060 : caputer ..
9061
9061 :

9375
9375 : caputer ..
9376
9376 : caputer ..
9377
9377 : caputer ..
9378
9378 : caputer ..
9379
9379 : caputer ..
9380
9380 : caputer ..
9381
9381 : caputer ..
9382
9382 : caputer ..
9383
9383 : caputer ..
9384
9384 : caputer ..
9385
9385 : caputer ..
9386
9386 : caputer ..
9387
9387 : caputer ..
9388
9388 : caputer ..
9389
9389 : caputer ..
9390
9390 : caputer ..
9391
9391 : caputer ..
9392
9392 : caputer ..
9393
9393 : caputer ..
9394
9394 : caputer ..
9395
9395 : caputer ..
9396
9396 : caputer ..
9397
9397 : caputer ..
9398
9398 : caputer ..
9399
9399 : caputer ..
9400
9400 : caputer ..
9401
9401 : caputer ..
9402
9402 : caputer ..
9403
9403 : caputer ..
9404
9404 : caputer ..
9405
9405 : caputer ..
9406
9406 : caputer ..
9407
9407 : caputer ..
9408
9408 : caputer ..
9409
9409 : caputer ..
9410
9410 : caputer ..
9411
9411 : caputer ..
9412
9412 : caputer ..
9413
9413 : caputer ..
9414
9414 : caputer ..
9415
9415 : caputer ..
9416
9416 : caputer ..
9417
9417 : caputer ..
9418
9418 :

9732
9732 : caputer ..
9733
9733 : caputer ..
9734
9734 : caputer ..
9735
9735 : caputer ..
9736
9736 : caputer ..
9737
9737 : caputer ..
9738
9738 : caputer ..
9739
9739 : caputer ..
9740
9740 : caputer ..
9741
9741 : caputer ..
9742
9742 : caputer ..
9743
9743 : caputer ..
9744
9744 : caputer ..
9745
9745 : caputer ..
9746
9746 : caputer ..
9747
9747 : caputer ..
9748
9748 : caputer ..
9749
9749 : caputer ..
9750
9750 : caputer ..
9751
9751 : caputer ..
9752
9752 : caputer ..
9753
9753 : caputer ..
9754
9754 : caputer ..
9755
9755 : caputer ..
9756
9756 : caputer ..
9757
9757 : caputer ..
9758
9758 : caputer ..
9759
9759 : caputer ..
9760
9760 : caputer ..
9761
9761 : caputer ..
9762
9762 : caputer ..
9763
9763 : caputer ..
9764
9764 : caputer ..
9765
9765 : caputer ..
9766
9766 : caputer ..
9767
9767 : caputer ..
9768
9768 : caputer ..
9769
9769 : caputer ..
9770
9770 : caputer ..
9771
9771 : caputer ..
9772
9772 : caputer ..
9773
9773 : caputer ..
9774
9774 : caputer ..
9775
9775 :

10082
10082 : caputer ..
10083
10083 : caputer ..
10084
10084 : caputer ..
10085
10085 : caputer ..
10086
10086 : caputer ..
10087
10087 : caputer ..
10088
10088 : caputer ..
10089
10089 : caputer ..
10090
10090 : caputer ..
10091
10091 : caputer ..
10092
10092 : caputer ..
10093
10093 : caputer ..
10094
10094 : caputer ..
10095
10095 : caputer ..
10096
10096 : caputer ..
10097
10097 : caputer ..
10098
10098 : caputer ..
10099
10099 : caputer ..
10100
10100 : caputer ..
10101
10101 : caputer ..
10102
10102 : caputer ..
10103
10103 : caputer ..
10104
10104 : caputer ..
10105
10105 : caputer ..
10106
10106 : caputer ..
10107
10107 : caputer ..
10108
10108 : caputer ..
10109
10109 : caputer ..
10110
10110 : caputer ..
10111
10111 : caputer ..
10112
10112 : caputer ..
10113
10113 : caputer ..
10114
10114 : caputer ..
10115
10115 : caputer ..
10116
10116 : caputer ..
10117
10117 : caputer ..
10118
10118 : caputer ..
10119
10119 : caputer ..
10120
10120 : caputer ..
10121
10121 : caputer ..
